In [2]:
pip install torch torchvision torchaudio

  Using cached torch-2.5.1-cp312-cp312-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-p

In [3]:
pip install scikit-learn

  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pandas

  Using cached pandas-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
Using cached pandas-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.7 MB)
Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install matplotlib

  Using cached matplotlib-3.10.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached contourpy-1.3.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached kiwisolver-1.4.8-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.2 kB)
  Using cached pyparsing-3.2.1-py3-none-any.whl.metadata (5.0 kB)
Using cached matplotlib-3.10.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (8.6 MB)
Using cached contourpy-1.3.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (323 kB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 57.1 MB/s eta 0:00:00
Using cached kiwisolver-1.4.8-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.5 MB)
Using cached pyparsing-3.2.1-py3-none-any.whl (107 kB)
Note: you may need to restart the kernel to use updated packages.

In [1]:
import torch
torch.cuda.is_available()


True

In [ ]:
import torch

# Force PyTorch to use CUDA even if ROCm is available
torch.cuda.set_device(0)  # Assuming device 0 is your CUDA-enabled device


# Prueba de PyTorch

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import mean_squared_error, median_absolute_error, r2_score
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt

# Dataframe
test_data = pd.read_csv("public_test.csv")
train_data = pd.read_csv("public_train.csv")

# Identificar columnas
TARGET = "ccs"
CATEGORICAL_FEATURES = ["adduct"]
NUMERIC_FEATURES = [col for col in train_data.columns if col.startswith("desc_")]
BINARY_FEATURES = [col for col in train_data.columns if col.startswith("fgp_")]

# Separar características y objetivo
x = train_data.drop(columns=[TARGET])  # Características
y = train_data[TARGET]  # Valor a estimar

# Dividir en datos de entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

class SimpleNN(BaseEstimator, RegressorMixin):
    def __init__(self, input_dim, hidden_dim=64, lr=0.001, epochs=100):
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.lr = lr
        self.epochs = epochs
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        if(torch.device == "cpu"):
            print("cpu")
        else:
            print("gpu")

        # Red neuronal
        self.model = nn.Sequential(
            nn.Linear(self.input_dim, self.hidden_dim),
            nn.ReLU(),
            nn.Linear(self.hidden_dim, 1)
        ).to(self.device)
        
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.lr)
        self.criterion = nn.MSELoss()

    def fit(self, X, y):
        # X y y son ahora ndarray
        X_tensor = torch.tensor(X, dtype=torch.float32).to(self.device)
        y_tensor = torch.tensor(y, dtype=torch.float32).view(-1, 1).to(self.device)

        for epoch in range(self.epochs):
            self.model.train()
            self.optimizer.zero_grad()
            predictions = self.model(X_tensor)
            loss = self.criterion(predictions, y_tensor)
            loss.backward()
            self.optimizer.step()

        return self

    def predict(self, X):
        self.model.eval()
        X_tensor = torch.tensor(X, dtype=torch.float32).to(self.device)
        with torch.no_grad():
            predictions = self.model(X_tensor)
        return predictions.cpu().numpy()


# Crear el modelo con los parámetros deseados
model_params = {'hidden_dim': 64, 'lr': 0.001, 'epochs': 40}

# Crear el pipeline de preprocesamiento y modelo
def create_preproc_and_model(numeric_features, categorical_features, binary_features):
    preprocessor = ColumnTransformer(
        transformers=[
            # Imputar valores faltantes y luego estandarizar los datos numéricos
            ('num', Pipeline([
                ('imputer', SimpleImputer(strategy='mean')),  # Usar la media para imputar
                ('scaler', StandardScaler())
            ]), numeric_features),

            # Imputar valores faltantes en los datos categóricos y aplicar OneHotEncoder
            ('cat', Pipeline([
                ('imputer', SimpleImputer(strategy='most_frequent')),  # Usar la moda para imputar
                ('onehot', OneHotEncoder())
            ]), categorical_features),

            # Imputar valores faltantes y luego estandarizar los datos binarios
            ('bin', Pipeline([
                ('imputer', SimpleImputer(strategy='most_frequent')),  # Usar la moda para imputar
                ('scaler', StandardScaler())
            ]), binary_features)
        ])
    
    # Ajusta el preprocesador a los datos de entrenamiento
    preprocessor.fit(x_train)
    
    # Verificar la forma de los datos después del preprocesamiento
    x_train_transformed = preprocessor.transform(x_train)
    
    input_dim = x_train_transformed.shape[1]  # Número de características después del preprocesamiento
    model = SimpleNN(input_dim=input_dim, hidden_dim=128, lr=0.00001, epochs=50000)

    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])

    return pipeline


pipeline = create_preproc_and_model(NUMERIC_FEATURES, CATEGORICAL_FEATURES, BINARY_FEATURES)

# Entrenar el modelo
pipeline.fit(x_train, y_train)

# Hacer predicciones
y_pred = pipeline.predict(x_test)

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse}")

medae = median_absolute_error(y_test, y_pred)
print(f"Median Absolute Error (MedAE): {medae}")

gpu
Mean Squared Error (MSE): 134.5720751569695
Median Absolute Error (MedAE): 3.9538956079101553


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import mean_squared_error, median_absolute_error, r2_score
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt

test_data = pd.read_csv("public_test.csv")
train_data = pd.read_csv("public_train.csv")

# Identificar columnas
TARGET = "ccs"
CATEGORICAL_FEATURES = ["adduct"]
NUMERIC_FEATURES = [col for col in train_data.columns if col.startswith("desc_")]
BINARY_FEATURES = [col for col in train_data.columns if col.startswith("fgp_")]

# Separar características y objetivo
x = train_data.drop(columns=[TARGET])  # Características
y = train_data[TARGET]

class SimpleNN(BaseEstimator, RegressorMixin):
    def __init__(self, input_dim, hidden_dim=64, lr=0.001, epochs=100):
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.lr = lr
        self.epochs = epochs
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        # Red neuronal
        self.model = nn.Sequential(
            nn.Linear(self.input_dim, self.hidden_dim),
            nn.ReLU(),
            nn.Linear(self.hidden_dim, 1)
        ).to(self.device)
        
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.lr)
        self.criterion = nn.MSELoss()
        self.loss_history = []

    def fit(self, X, y):
        X_tensor = torch.tensor(X, dtype=torch.float32).to(self.device)
        y_tensor = torch.tensor(y, dtype=torch.float32).view(-1, 1).to(self.device)

        for epoch in range(self.epochs):
            self.model.train()
            self.optimizer.zero_grad()
            predictions = self.model(X_tensor)
            loss = self.criterion(predictions, y_tensor)
            loss.backward()
            self.optimizer.step()

            # Guardar el MSE de cada época
            self.loss_history.append(loss.item())

        return self

    def predict(self, X):
        self.model.eval()
        X_tensor = torch.tensor(X, dtype=torch.float32).to(self.device)
        with torch.no_grad():
            predictions = self.model(X_tensor)
        return predictions.cpu().numpy()

# Crear el pipeline de preprocesamiento y modelo
def create_preproc_and_model(numeric_features, categorical_features, binary_features):
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', Pipeline([
                ('imputer', SimpleImputer(strategy='mean')),
                ('scaler', StandardScaler())
            ]), numeric_features),
            ('cat', Pipeline([
                ('imputer', SimpleImputer(strategy='most_frequent')),
                ('onehot', OneHotEncoder())
            ]), categorical_features),
            ('bin', Pipeline([
                ('imputer', SimpleImputer(strategy='most_frequent')),
                ('scaler', StandardScaler())
            ]), binary_features)
        ])
    
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

    preprocessor.fit(x_train)
    x_train_transformed = preprocessor.transform(x_train)
    input_dim = x_train_transformed.shape[1]
    model = SimpleNN(input_dim=input_dim, hidden_dim=1028, lr=0.00002, epochs=10000)

    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])

    return pipeline, x_train, x_test, y_train, y_test

pipeline, x_train, x_test, y_train, y_test = create_preproc_and_model(NUMERIC_FEATURES, CATEGORICAL_FEATURES, BINARY_FEATURES)

# Entrenar el modelo
pipeline.fit(x_train, y_train)

# Graficar la evolución del MSE
model = pipeline.named_steps['model']  # Obtener el modelo entrenado
plt.plot(model.loss_history)
plt.xlabel('Épocas')
plt.ylabel('MSE')
plt.title('Evolución del MSE durante el entrenamiento')
plt.show()

# Hacer predicciones
y_pred = pipeline.predict(x_test)

# Calcular y mostrar MSE y MedAE
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse}")

medae = median_absolute_error(y_test, y_pred)
print(f"Median Absolute Error (MedAE): {medae}")

r2 = r2_score(y_test, y_pred)
print("R² Score:", r2)



### Creamos más capas + Regulaciones + Drop Out

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, RandomizedSearchCV
import pandas as pd
from sklearn.metrics import mean_squared_error, median_absolute_error, r2_score
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import numpy as np



test_data = pd.read_csv("public_test.csv")
train_data = pd.read_csv("public_train.csv")

# Identificar columnas
TARGET = "ccs"
CATEGORICAL_FEATURES = ["adduct"]
NUMERIC_FEATURES = [col for col in train_data.columns if col.startswith("desc_")]
BINARY_FEATURES = [col for col in train_data.columns if col.startswith("fgp_")]

# Separar características y objetivo
x = train_data.drop(columns=[TARGET])  # Características
y = train_data[TARGET]

class SimpleNN(BaseEstimator, RegressorMixin):
    def __init__(self, input_dim, hidden_dims=[64, 32, 16], lr=0.001, epochs=100, batch_size=32, activation='ReLU', dropout=0.0):
        self.input_dim = input_dim
        self.hidden_dims = hidden_dims
        self.lr = lr
        self.epochs = epochs
        self.batch_size = batch_size
        self.activation = activation
        self.dropout = dropout
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Definir las capas de la red neuronal
        layers = []
        prev_dim = self.input_dim

        for hidden_dim in self.hidden_dims:
            layers.append(nn.Linear(prev_dim, hidden_dim))
            if self.activation == 'ReLU':
                layers.append(nn.ReLU())
            elif self.activation == 'Tanh':
                layers.append(nn.Tanh())
            elif self.activation == 'Sigmoid':
                layers.append(nn.Sigmoid())
            elif self.activation == 'LeakyReLU':
                layers.append(nn.LeakyReLU())
            if self.dropout > 0:
                layers.append(nn.Dropout(self.dropout))
            prev_dim = hidden_dim

        # Capa de salida
        layers.append(nn.Linear(prev_dim, 1))

        # Crear el modelo secuencial
        self.model = nn.Sequential(*layers).to(self.device)

        self.optimizer = optim.Adam(self.model.parameters(), lr=self.lr)
        self.criterion = nn.MSELoss()
        self.loss_history = []

    def fit(self, X, y):
        X_tensor = torch.tensor(X, dtype=torch.float32).to(self.device)
        y_tensor = torch.tensor(y, dtype=torch.float32).view(-1, 1).to(self.device)

        dataset = torch.utils.data.TensorDataset(X_tensor, y_tensor)
        dataloader = torch.utils.data.DataLoader(dataset, batch_size=self.batch_size, shuffle=True)

        for epoch in range(self.epochs):
            self.model.train()
            epoch_loss = 0
            for X_batch, y_batch in dataloader:
                self.optimizer.zero_grad()
                predictions = self.model(X_batch)
                loss = self.criterion(predictions, y_batch)
                loss.backward()
                self.optimizer.step()
                epoch_loss += loss.item()
            self.loss_history.append(epoch_loss / len(dataloader))

        return self

    def predict(self, X):
        self.model.eval()
        X_tensor = torch.tensor(X, dtype=torch.float32).to(self.device)
        with torch.no_grad():
            predictions = self.model(X_tensor)
        return predictions.cpu().numpy()

# Crear el pipeline de preprocesamiento y modelo
# Crear el pipeline de preprocesamiento y modelo
def create_preproc_and_model(numeric_features, categorical_features, binary_features):
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', Pipeline([
                ('imputer', SimpleImputer(strategy='mean')),
                ('scaler', StandardScaler())
            ]), numeric_features),
            ('cat', Pipeline([
                ('imputer', SimpleImputer(strategy='most_frequent')),
                ('onehot', OneHotEncoder())
            ]), categorical_features),
            ('bin', Pipeline([
                ('imputer', SimpleImputer(strategy='most_frequent')),
                ('scaler', StandardScaler())
            ]), binary_features)
        ])
    
    # Dividir los datos en entrenamiento y prueba
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

    # Asegúrate de ajustar el preprocesador antes de cualquier transformación
    preprocessor.fit(x_train)

    # Transformar los datos de entrenamiento
    x_train_transformed = preprocessor.transform(x_train)
    input_dim = x_train_transformed.shape[1]

    # Crear el modelo
    model = SimpleNN(input_dim=input_dim ,lr = 0.0009, hidden_dims= [64, 32, 16], epochs = 500, dropout = 0.2, batch_size = 64, activation = 'ReLU')

    # Crear el pipeline
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])

    return pipeline, x_train, x_test, y_train, y_test

# Crear el pipeline
pipeline, x_train, x_test, y_train, y_test = create_preproc_and_model(NUMERIC_FEATURES, CATEGORICAL_FEATURES, BINARY_FEATURES)

# Entrenar el modelo
pipeline.fit(x_train, y_train)

# Graficar la evolución del MSE
model = pipeline.named_steps['model']  # Obtener el modelo entrenado
plt.plot(model.loss_history)
plt.xlabel('Épocas')
plt.ylabel('MSE')
plt.title('Evolución del MSE durante el entrenamiento')
plt.show()

# Hacer predicciones
y_pred = pipeline.predict(x_test)

# Calcular y mostrar MSE y MedAE
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse}")

medae = median_absolute_error(y_test, y_pred)
print(f"Median Absolute Error (MedAE): {medae}")

r2 = r2_score(y_test, y_pred)
print("R² Score:", r2)


ModuleNotFoundError: No module named 'joblib'

### Prueba de parámetros

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, median_absolute_error, r2_score, make_scorer
from sklearn.impute import SimpleImputer
import pandas as pd
import matplotlib.pyplot as plt

# Leer los datos
test_data = pd.read_csv("public_test.csv")
train_data = pd.read_csv("public_train.csv")

# Identificar columnas
TARGET = "ccs"
CATEGORICAL_FEATURES = ["adduct"]
NUMERIC_FEATURES = [col for col in train_data.columns if col.startswith("desc_")]
BINARY_FEATURES = [col for col in train_data.columns if col.startswith("fgp_")]

# Separar características y objetivo
x = train_data.drop(columns=[TARGET])  # Características
y = train_data[TARGET]

# Definir el modelo
class SimpleNN(BaseEstimator, RegressorMixin):
    def __init__(self, input_dim, hidden_dims=[64, 32, 16], lr=0.001, epochs=100, batch_size=32, activation='ReLU', dropout=0.0):
        self.input_dim = input_dim
        self.hidden_dims = hidden_dims
        self.lr = lr
        self.epochs = epochs
        self.batch_size = batch_size
        self.activation = activation
        self.dropout = dropout
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        if self.device.type == "cpu":
            print("Using CPU")
        else:
            print("Using GPU")

        layers = []
        prev_dim = self.input_dim
        for hidden_dim in self.hidden_dims:
            layers.append(nn.Linear(prev_dim, hidden_dim))
            layers.append(getattr(nn, self.activation)())
            if self.dropout > 0:
                layers.append(nn.Dropout(self.dropout))
            prev_dim = hidden_dim

        layers.append(nn.Linear(prev_dim, 1))
        self.model = nn.Sequential(*layers).to(self.device)

        self.optimizer = optim.Adam(self.model.parameters(), lr=self.lr)
        self.criterion = nn.MSELoss()
        self.loss_history = []

    def fit(self, X, y):
        X_tensor = torch.tensor(X, dtype=torch.float32).to(self.device)
        y_tensor = torch.tensor(y, dtype=torch.float32).view(-1, 1).to(self.device)

        dataset = torch.utils.data.TensorDataset(X_tensor, y_tensor)
        dataloader = torch.utils.data.DataLoader(dataset, batch_size=self.batch_size, shuffle=True)

        for epoch in range(self.epochs):
            self.model.train()
            epoch_loss = 0
            for X_batch, y_batch in dataloader:
                self.optimizer.zero_grad()
                predictions = self.model(X_batch)
                loss = self.criterion(predictions, y_batch)
                loss.backward()
                self.optimizer.step()
                epoch_loss += loss.item()
            self.loss_history.append(epoch_loss / len(dataloader))

        return self

    def predict(self, X):
        self.model.eval()
        X_tensor = torch.tensor(X, dtype=torch.float32).to(self.device)
        with torch.no_grad():
            predictions = self.model(X_tensor)
        return predictions.cpu().numpy()

# Crear el pipeline de preprocesamiento y modelo
def create_preproc_and_model(numeric_features, categorical_features, binary_features):
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', Pipeline([
                ('imputer', SimpleImputer(strategy='mean')),
                ('scaler', StandardScaler())
            ]), numeric_features),
            ('cat', Pipeline([
                ('imputer', SimpleImputer(strategy='most_frequent')),
                ('onehot', OneHotEncoder())
            ]), categorical_features),
            ('bin', Pipeline([
                ('imputer', SimpleImputer(strategy='most_frequent')),
                ('scaler', StandardScaler())
            ]), binary_features)
        ]
    )

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
    preprocessor.fit(x_train)
    x_train_transformed = preprocessor.transform(x_train)
    input_dim = x_train_transformed.shape[1]

    # Convertir x_train y x_test a DataFrames para que ColumnTransformer pueda funcionar
    x_train = pd.DataFrame(x_train, columns=train_data.drop(columns=[TARGET]).columns)
    x_test = pd.DataFrame(x_test, columns=train_data.drop(columns=[TARGET]).columns)

    model = SimpleNN(input_dim=input_dim, hidden_dims=[128, 64, 32], lr=0.0001, epochs=500, batch_size=64, activation='LeakyReLU', dropout=0.3)

    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])

    return pipeline, x_train, x_test, y_train, y_test

# Crear y configurar el RandomizedSearchCV
pipeline, x_train, x_test, y_train, y_test = create_preproc_and_model(NUMERIC_FEATURES, CATEGORICAL_FEATURES, BINARY_FEATURES)

param_dist = {
    'model__hidden_dims': [[128, 64, 32], [64, 32, 16], [256, 128, 64]],
    'model__lr': [0.01, 0.001, 0.0001, 0.00001],
    'model__epochs': [100, 500, 1000],
    'model__batch_size': [32, 64, 128],
    'model__activation': ['ReLU', 'LeakyReLU'],
    'model__dropout': [0.0, 0.2, 0.3, 0.5]
}

medae_scorer = make_scorer(median_absolute_error, greater_is_better=False)

random_search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_dist,
    n_iter=10,
    cv=3,
    random_state=42,
    n_jobs=-1,
    scoring=medae_scorer
)

random_search.fit(x_train, y_train)

print("Mejores parámetros encontrados:", random_search.best_params_)

y_pred = random_search.predict(x_test)

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse}")

medae = median_absolute_error(y_test, y_pred)
print(f"Median Absolute Error (MedAE): {medae}")

r2 = r2_score(y_test, y_pred)
print("R² Score:", r2)

# Graficar la evolución del MSE
best_model = random_search.best_estimator_.named_steps['model']
plt.plot(best_model.loss_history)
plt.xlabel('Épocas')
plt.ylabel('MSE')
plt.title('Evolución del MSE durante el entrenamiento')
plt.show()


Investigar torch.profiler y DataLoader